##### Copyright 2019 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Autoencoder - Tensorflow 2.0 with eager mode and graph mode

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/template/notebook.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/template/notebook.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

This notebook demonstrates how to generate images of handwritten digits using both eager execution and graph mode by training an Autoencoder.


## Setup

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

try:
  # The %tensorflow_version magic only works in colab.
  %tensorflow_version 2.x
except Exception:
  pass
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.__version__)

In [0]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow.keras.layers as layers

## Preprocessing functions

In [0]:
def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test

In [0]:
def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data) - batch_size)
    return data[start_index:(start_index + batch_size)]

## Autoencoder model

In [0]:
class Encoder(tf.keras.layers.Layer):
    '''Encodes a digit from the MNIST dataset'''
    
    def __init__(self,
                n_dims,
                name='encoder',
                **kwargs):
        super(Encoder,self).__init__(name=name, **kwargs)
        self.n_dims = n_dims
        self.n_layers = 1
        self.encode_layer = layers.Dense(n_dims, activation='relu')
        
    @tf.function        
    def call(self, inputs):
        return self.encode_layer(inputs)

In [0]:
class Decoder(tf.keras.layers.Layer):
    '''Decodes a digit from the MNIST dataset'''

    def __init__(self,
                n_dims,
                name='decoder',
                **kwargs):
        super(Decoder,self).__init__(name=name, **kwargs)
        self.n_dims = n_dims
        self.n_layers = len(n_dims)
        self.decode_middle = layers.Dense(n_dims[0], activation='relu')
        self.recon_layer = layers.Dense(n_dims[1], activation='sigmoid')
        
    @tf.function        
    def call(self, inputs):
        x = self.decode_middle(inputs)
        return self.recon_layer(x)

In [0]:
class Autoencoder(tf.keras.Model):
    '''Vanilla Autoencoder for MNIST digits'''
    
    def __init__(self,
                 n_dims=[200, 392, 784],
                 name='autoencoder',
                 **kwargs):
        super(Autoencoder, self).__init__(name=name, **kwargs)
        self.n_dims = n_dims
        self.encoder = Encoder(n_dims[0])
        self.decoder = Decoder([n_dims[1], n_dims[2]])
        
    @tf.function        
    def call(self, inputs):
        x = self.encoder(inputs)
        return self.decoder(x)

## Loading the dataset

In [0]:
mnist = tf.keras.datasets.mnist

In [0]:
(X_train, _), (X_test, _) = mnist.load_data()
X_train = tf.cast(np.reshape(X_train, (X_train.shape[0], X_train.shape[1] * X_train.shape[2])), tf.float64)
X_test = tf.cast(np.reshape(X_test, (X_test.shape[0], X_test.shape[1] * X_test.shape[2])), tf.float64)

X_train, X_test = standard_scale(X_train, X_test)

In [0]:
train_data = tf.data.Dataset.from_tensor_slices(X_train).batch(128).shuffle(buffer_size=1024)
test_data = tf.data.Dataset.from_tensor_slices(X_test).batch(128).shuffle(buffer_size=512)

## Training setup

In [0]:
n_samples = int(len(X_train) + len(X_test))
training_epochs = 20
batch_size = 128
display_step = 1

In [0]:
optimizer = tf.optimizers.Adam(learning_rate=0.01)
mse_loss = tf.keras.losses.MeanSquaredError()
loss_metric = tf.keras.metrics.Mean()

## Eager mode training
Using `tf.GradientTape()` to expose the tape to perform Autodifferentiation. Tensorflow "records" all operations executed inside the context of a "tape". Tensorflow then uses that tape and the gradients associated with each recorded operation to compute the gradients of a "recorded" computation using reverse mode differentiation. The optimizer can then be used to apply the gradients to all the trainable parameters.

In [0]:
autoencoder = Autoencoder([200, 394, 784])

In [0]:
# Iterate over epochs.
for epoch in range(5):
    print(f'Epoch {epoch+1}')

  # Iterate over the batches of the dataset.
    for step, x_batch in enumerate(train_data):
        with tf.GradientTape() as tape:
          recon = autoencoder(x_batch)
          loss = mse_loss(x_batch, recon)

        grads = tape.gradient(loss, autoencoder.trainable_variables)
        optimizer.apply_gradients(zip(grads, autoencoder.trainable_variables))

        loss_metric(loss)

        if step % 100 == 0:
          print(f'Step {step}: mean loss = {loss_metric.result()}')

## Graph mode training
Graph mode training with `tf.keras.Model.fit` creates a computation graph and fits the training data on the keras model.

In [0]:
ae = Autoencoder([200, 392, 784])
ae.compile(optimizer=tf.optimizers.Adam(0.01), loss='categorical_crossentropy')
ae.fit(X_train, X_train, batch_size=64, epochs=5)

## Notes

- Eager Execution Basics [TF Docs Link](https://www.tensorflow.org/tutorials/eager/eager_basics)
- Tensorflow Automatic Differentiation [TF Docs Link](https://www.tensorflow.org/tutorials/eager/automatic_differentiation)